In [180]:
import pandas as pd
import numpy as np

In [181]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

In [182]:
from glob import glob

## 데이터 불러오기

In [451]:
w_list = sorted(glob("competition_data/water_data/*.csv"))
r_list = sorted(glob("competition_data/rf_data/*.csv"))
sample=pd.read_csv('competition_data/sample_submission.csv')
total_wd=pd.DataFrame()
for i in range(0, len(w_list)):
    total_wd=pd.concat([total_wd, pd.read_csv(w_list[i])])
total_rf=pd.DataFrame()
for i in range(0, len(r_list)):
    total_rf=pd.concat([total_rf, pd.read_csv(r_list[i])])

In [452]:
df_rf=total_rf.copy()
df_water=total_wd.copy()
submission=sample.copy()

In [453]:
df_rf.set_index('ymdhm', inplace=True)
df_water.set_index('ymdhm', inplace=True)
submission.set_index('ymdhm', inplace=True)

In [454]:
df_rf.name = "rain_data"
df_water.name = "water_data"
submission.name = "submission"

# 데이터 전처리

## 기본 세팅 (data, target 분리)

In [455]:
def index_to_datetime(df,format):
    df.index = pd.to_datetime(df.index,
                              format=format)
    return df

In [456]:
df_rf=index_to_datetime(df=df_rf,format='%Y-%m-%d %H:%M')
df_water=index_to_datetime(df=df_water,format='%Y-%m-%d %H:%M')
submission=index_to_datetime(df=submission,format='%Y-%m-%d %H:%M')

In [457]:
df_rf.sort_index(inplace=True)
df_water.sort_index(inplace=True)
submission.sort_index(inplace=True)

In [458]:
# 데이터 시간대 확인하기
def check_datetime(df):
    print(df.name)
    print(df.select_dtypes('datetime64[ns]').head(1).index[0])
    print(df.select_dtypes('datetime64[ns]').tail(1).index[0])
    return None

check_datetime(df_rf)
check_datetime(df_water)
check_datetime(submission)

rain_data
2012-05-01 00:00:00
2022-07-18 23:50:00
water_data
2012-05-01 00:00:00
2022-07-18 23:50:00
submission
2022-06-01 00:00:00
2022-07-18 23:50:00


In [459]:
df_rf_copy=df_rf.copy()
df_water_copy=df_water.copy()

In [460]:
# data target 분리하기
target = df_water.loc[:,submission.columns]
data = pd.concat((df_rf,df_water.drop(submission.columns,axis=1)),axis=1)

## 날씨데이터

In [461]:
# weather_df
weather_df=pd.read_csv('weather_df.csv')

In [462]:
weather_df['date']=pd.to_datetime(weather_df['date'])

In [463]:
weather_df=weather_df[['date','ta','rn','ws','wd','hm','pv','td','pa','ps']]

In [464]:
for i in weather_df.columns[1:]:
    weather_df.loc[:,i]=weather_df.loc[:,i].interpolate()

In [465]:
weather_df['rn']=weather_df['rn'].interpolate()

In [466]:
weather_df['rn']=weather_df['rn'].fillna(weather_df['rn'].mean())

In [467]:
def make_datetime(df):
    df['year']=df['date'].dt.year
    df['month']=df['date'].dt.month
    df['day']=df['date'].dt.day
    df['hour']=df['date'].dt.hour
    return df

In [468]:
def make_datetime2(df):
    df['year']=df['date'].dt.year
    df['month']=df['date'].dt.month
    df['day']=df['date'].dt.day
    df['hour']=df['date'].dt.hour
    df['minute']=df['date'].dt.minute
    return df

In [469]:
weather_df=make_datetime(weather_df)

In [470]:
data.reset_index(inplace=True)

In [471]:
data['date']=pd.to_datetime(data['ymdhm'])

In [472]:
data=make_datetime2(data)

In [473]:
data.drop(['date'], axis=1, inplace=True)
weather_df.drop(['date'], axis=1, inplace=True)

In [474]:
data=pd.merge(data, weather_df, how='left', on=['year','month','day','hour'])

## dam_api

In [475]:
dam_df=pd.read_csv('dam_api.csv')
dam_df['date']=pd.to_datetime(dam_df['ymdhm'], format="%Y%m%d%H%M")
dam_df=dam_df.sort_values(by='date')
from datetime import timedelta
# 10분씩 밀어줌
dam_df['date']=dam_df['date']+timedelta(minutes=10)
dam_df.drop('ymdhm', axis=1, inplace=True)

C:\Users\JiyoonLee\AppData\Local\Temp\ipykernel_3324\1310388657.py:1: DtypeWarning: Columns (1,2,3,4,5,6,7,8,11,12,13,14,15,16,17,18,19,20,21,23,24,25,26,27,28) have mixed types. Specify dtype option on import or set low_memory=False.
  dam_df=pd.read_csv('dam_api.csv')


In [476]:
dam_df.sort_values(by='date')
dam_df.reset_index(inplace=True, drop=True)

In [477]:
dam_df=dam_df.replace(" ", np.nan)
dam_df=dam_df.replace("", np.nan)

In [478]:
# 전체가 null 값인 데이터 + # 청담대교 수위 잠수교 수위 한강대교 수위 행주대교 수위 wl 제외
remove_list=['서울시(창동교)_fw','팔당대교_wl','팔당대교_fw', '서울시(행주대교)_wl', '서울시(한강대교)_wl', '서울시(청담대교)_wl', '서울시(잠수교)_wl']
columns_list=list(dam_df.columns)

In [479]:
final_list = list(set(dam_df.columns)-set(remove_list))

In [480]:
dam_df=dam_df[final_list]

In [481]:
for i in dam_df.columns:
    if i=='date':
        pass
    else:
        dam_df[i]=dam_df[i].astype('float64')

In [482]:
for i in dam_df.columns:
    if i =='date':
        pass
    else:
        dam_df[i]=dam_df[i].interpolate()

In [483]:
for i in dam_df.columns:
    dam_df[i]=dam_df[i].fillna(dam_df[i].mean())

In [484]:
dam_df=make_datetime2(dam_df)
dam_df.drop('date', axis=1, inplace=True)

In [485]:
data.shape, dam_df.shape

((276336, 28), (278208, 30))

In [486]:
data=pd.merge(data, dam_df, how='left', on=['year', 'month','day','hour','minute'])

In [487]:
data.set_index('ymdhm', inplace=True)

In [488]:
data.drop(['year','month','day','hour'], axis=1, inplace=True)

In [489]:
data.isnull().sum()

rf_10184100           0
rf_10184110           0
rf_10184140           0
swl                 743
inf                 743
sfw                 743
ecpc                743
tototf              743
tide_level         4927
fw_1018662        16380
fw_1018680       196849
fw_1018683         1279
fw_1019630           59
minute                0
ta                    0
rn                    0
ws                    0
wd                    0
hm                    0
pv                    0
td                    0
pa                    0
ps                    0
팔당댐_wl               11
남양주시(팔당대교)_wl        11
서울시(너부대교)_fw         11
남양주시(팔당대교)_fw        11
남양주시(팔당댐)_wl         11
서울시(오금교)_wl          11
서울시(너부대교)_wl         11
남양주시(팔당댐)_fw         11
서울시(월계2교)_wl         11
서울시(광진교)_wl          11
서울시(대곡교)_wl          11
서울시(대곡교)_fw          11
서울시(광진교)_fw          11
서울시(월계2교)_fw         11
서울시(중랑교)_wl          11
서울시(창동교)_wl          11
서울시(대치교)_fw          11
서울시(대치교)_wl          11
서울시(청담대교)_fw    

In [490]:
for i in data.columns[23:]:
    data[i]=data[i].fillna(method='bfill')

## taraget, data, 인덱스 밀어주기

In [493]:
# # data와 target 하나 밀어주기 (과거데이터를 사용해야 함으로)
_target = target.reset_index(drop=True)
_data = data.reset_index(drop=True)


In [494]:
_data.index+=1

In [495]:
_data

,rf_10184100,rf_10184110,rf_10184140,swl,inf,sfw,ecpc,tototf,tide_level,fw_1018662,fw_1018680,fw_1018683,fw_1019630,minute,ta,rn,ws,wd,hm,pv,td,pa,ps,팔당댐_wl,남양주시(팔당대교)_wl,서울시(너부대교)_fw,남양주시(팔당대교)_fw,남양주시(팔당댐)_wl,서울시(오금교)_wl,서울시(너부대교)_wl,남양주시(팔당댐)_fw,서울시(월계2교)_wl,서울시(광진교)_wl,서울시(대곡교)_wl,서울시(대곡교)_fw,서울시(광진교)_fw,서울시(월계2교)_fw,서울시(중랑교)_wl,서울시(창동교)_wl,서울시(대치교)_fw,서울시(대치교)_wl,서울시(청담대교)_fw,서울시(행주대교)_fw,팔당댐_fw,서울시(잠수교)_fw,서울시(한강대교)_fw,서울시(오금교)_fw,서울시(중랑교)_fw
1,0.0,0.0,0.0,24.800,555.00,219.07,24.93,555.00,445.0,469.05,0.0,729.80,540.18,0,21.2,0.426115,2.4,110.0,43.0,10.8,8.1,1001.1,1011.0,0.0,1.56,1.12,497.47,24.81,1.06,0.17,0.0,0.44,1.20,1.78,7.67,2054.51,0.42,0.54,1.212996,2.713208,1.537687,469.05,540.18,0.0,0.0,729.80,6.71,3.88
2,0.0,0.0,0.0,24.794,464.60,218.86,25.15,562.90,449.0,498.00,0.0,731.48,540.18,10,21.2,0.426115,2.4,110.0,43.0,10.8,8.1,1001.1,1011.0,0.0,1.56,1.12,497.47,24.81,1.06,0.17,0.0,0.44,1.20,1.78,7.67,2054.51,0.42,0.54,1.212996,2.713208,1.537687,469.05,540.18,0.0,0.0,729.80,6.71,3.88
3,0.0,0.0,0.0,24.789,478.10,218.69,25.31,576.40,451.0,490.68,0.0,726.42,540.18,20,21.2,0.426115,2.4,110.0,43.0,10.8,8.1,1001.1,1011.0,0.0,1.56,1.12,497.47,24.81,1.06,0.17,0.0,0.44,1.20,1.78,7.67,2054.51,0.42,0.54,1.212996,2.713208,1.537687,498.00,540.18,0.0,0.0,731.48,6.71,3.88
4,0.0,0.0,0.0,24.789,464.80,218.69,25.31,563.10,452.0,476.21,0.0,726.42,552.17,30,21.2,0.426115,2.4,110.0,43.0,10.8,8.1,1001.1,1011.0,0.0,1.56,1.12,497.47,24.80,1.06,0.17,0.0,0.43,1.20,1.78,7.67,2054.51,0.36,0.54,1.212996,2.713208,1.537687,490.68,540.18,0.0,0.0,726.42,6.71,3.88
5,0.0,0.0,0.0,24.789,478.10,218.69,25.31,576.40,450.0,476.21,0.0,707.17,564.29,40,21.2,0.426115,2.4,110.0,43.0,10.8,8.1,1001.1,1011.0,0.0,1.56,1.12,497.47,24.80,1.06,0.17,0.0,0.44,1.20,1.78,7.67,2054.51,0.42,0.54,1.212996,2.713208,1.537687,476.21,552.17,0.0,0.0,726.42,6.71,3.88
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
276332,0.0,0.0,0.0,25.040,259.23,212.86,31.14,259.23,510.0,319.84,NaN,-456.41,974.40,10,26.2,0.000000,1.9,250.0,78.0,26.4,22.0,989.5,999.1,0.0,1.18,1.35,255.36,25.03,0.85,0.20,0.0,0.34,1.12,2.21,3.13,282.85,0.62,0.69,0.950000,1.390000,1.560000,319.84,926.70,0.0,0.0,25.27,6.64,8.66
276333,0.0,0.0,0.0,25.040,260.46,212.86,31.14,260.46,492.0,314.01,NaN,-717.30,1006.88,20,26.2,0.000000,1.9,250.0,78.0,26.4,22.0,989.5,999.1,0.0,1.18,1.35,255.36,25.03,0.85,0.20,0.0,0.34,1.12,2.21,2.77,315.64,0.62,0.68,0.950000,1.390000,1.560000,319.84,974.40,0.0,0.0,-456.41,6.64,8.33
276334,0.0,0.0,0.0,25.040,259.37,212.86,31.14,259.37,475.0,387.55,NaN,-843.37,1039.90,30,26.2,0.000000,1.9,250.0,78.0,26.4,22.0,989.5,999.1,0.0,1.18,1.35,255.36,25.03,0.85,0.20,0.0,0.34,1.12,2.21,4.01,348.44,0.62,0.68,0.950000,1.390000,1.560000,314.01,1006.88,0.0,0.0,-717.30,6.64,8.33
276335,0.0,0.0,0.0,25.040,259.13,212.86,31.14,259.13,458.0,454.91,NaN,-1023.37,1073.46,40,26.2,0.000000,1.9,250.0,78.0,26.4,22.0,989.5,999.1,0.0,1.18,1.35,255.36,25.04,0.85,0.20,0.0,0.34,1.12,2.21,3.54,319.74,0.62,0.68,0.950000,1.390000,1.560000,387.55,1039.90,0.0,0.0,-843.37,6.64,8.33


In [496]:
_target

,wl_1018662,wl_1018680,wl_1018683,wl_1019630
0,310.7,300.2,290.0,275.3
1,314.7,300.2,290.0,275.3
2,313.7,301.2,290.0,275.3
3,311.7,301.2,290.0,276.3
4,311.7,301.2,291.0,277.3
...,...,...,...,...
276331,0.0,0.0,0.0,0.0
276332,0.0,0.0,0.0,0.0
276333,0.0,0.0,0.0,0.0
276334,0.0,0.0,0.0,0.0


In [497]:
tot=pd.concat((_data,_target),axis=1)

In [498]:
tot
# 0인덱스에 data 값은 nan 값이 들어감

,rf_10184100,rf_10184110,rf_10184140,swl,inf,sfw,ecpc,tototf,tide_level,fw_1018662,fw_1018680,fw_1018683,fw_1019630,minute,ta,rn,ws,wd,hm,pv,td,pa,ps,팔당댐_wl,남양주시(팔당대교)_wl,서울시(너부대교)_fw,남양주시(팔당대교)_fw,남양주시(팔당댐)_wl,서울시(오금교)_wl,서울시(너부대교)_wl,남양주시(팔당댐)_fw,서울시(월계2교)_wl,서울시(광진교)_wl,서울시(대곡교)_wl,서울시(대곡교)_fw,서울시(광진교)_fw,서울시(월계2교)_fw,서울시(중랑교)_wl,서울시(창동교)_wl,서울시(대치교)_fw,서울시(대치교)_wl,서울시(청담대교)_fw,서울시(행주대교)_fw,팔당댐_fw,서울시(잠수교)_fw,서울시(한강대교)_fw,서울시(오금교)_fw,서울시(중랑교)_fw,wl_1018662,wl_1018680,wl_1018683,wl_1019630
1,0.0,0.0,0.0,24.800,555.00,219.07,24.93,555.00,445.0,469.05,0.0,729.80,540.18,0.0,21.2,0.426115,2.4,110.0,43.0,10.8,8.1,1001.1,1011.0,0.0,1.56,1.12,497.47,24.81,1.06,0.17,0.0,0.44,1.20,1.78,7.67,2054.51,0.42,0.54,1.212996,2.713208,1.537687,469.05,540.18,0.0,0.0,729.80,6.71,3.88,314.7,300.2,290.0,275.3
2,0.0,0.0,0.0,24.794,464.60,218.86,25.15,562.90,449.0,498.00,0.0,731.48,540.18,10.0,21.2,0.426115,2.4,110.0,43.0,10.8,8.1,1001.1,1011.0,0.0,1.56,1.12,497.47,24.81,1.06,0.17,0.0,0.44,1.20,1.78,7.67,2054.51,0.42,0.54,1.212996,2.713208,1.537687,469.05,540.18,0.0,0.0,729.80,6.71,3.88,313.7,301.2,290.0,275.3
3,0.0,0.0,0.0,24.789,478.10,218.69,25.31,576.40,451.0,490.68,0.0,726.42,540.18,20.0,21.2,0.426115,2.4,110.0,43.0,10.8,8.1,1001.1,1011.0,0.0,1.56,1.12,497.47,24.81,1.06,0.17,0.0,0.44,1.20,1.78,7.67,2054.51,0.42,0.54,1.212996,2.713208,1.537687,498.00,540.18,0.0,0.0,731.48,6.71,3.88,311.7,301.2,290.0,276.3
4,0.0,0.0,0.0,24.789,464.80,218.69,25.31,563.10,452.0,476.21,0.0,726.42,552.17,30.0,21.2,0.426115,2.4,110.0,43.0,10.8,8.1,1001.1,1011.0,0.0,1.56,1.12,497.47,24.80,1.06,0.17,0.0,0.43,1.20,1.78,7.67,2054.51,0.36,0.54,1.212996,2.713208,1.537687,490.68,540.18,0.0,0.0,726.42,6.71,3.88,311.7,301.2,291.0,277.3
5,0.0,0.0,0.0,24.789,478.10,218.69,25.31,576.40,450.0,476.21,0.0,707.17,564.29,40.0,21.2,0.426115,2.4,110.0,43.0,10.8,8.1,1001.1,1011.0,0.0,1.56,1.12,497.47,24.80,1.06,0.17,0.0,0.44,1.20,1.78,7.67,2054.51,0.42,0.54,1.212996,2.713208,1.537687,476.21,552.17,0.0,0.0,726.42,6.71,3.88,311.7,301.2,291.0,277.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
276333,0.0,0.0,0.0,25.040,260.46,212.86,31.14,260.46,492.0,314.01,NaN,-717.30,1006.88,20.0,26.2,0.000000,1.9,250.0,78.0,26.4,22.0,989.5,999.1,0.0,1.18,1.35,255.36,25.03,0.85,0.20,0.0,0.34,1.12,2.21,2.77,315.64,0.62,0.68,0.950000,1.390000,1.560000,319.84,974.40,0.0,0.0,-456.41,6.64,8.33,0.0,0.0,0.0,0.0
276334,0.0,0.0,0.0,25.040,259.37,212.86,31.14,259.37,475.0,387.55,NaN,-843.37,1039.90,30.0,26.2,0.000000,1.9,250.0,78.0,26.4,22.0,989.5,999.1,0.0,1.18,1.35,255.36,25.03,0.85,0.20,0.0,0.34,1.12,2.21,4.01,348.44,0.62,0.68,0.950000,1.390000,1.560000,314.01,1006.88,0.0,0.0,-717.30,6.64,8.33,0.0,0.0,0.0,0.0
276335,0.0,0.0,0.0,25.040,259.13,212.86,31.14,259.13,458.0,454.91,NaN,-1023.37,1073.46,40.0,26.2,0.000000,1.9,250.0,78.0,26.4,22.0,989.5,999.1,0.0,1.18,1.35,255.36,25.04,0.85,0.20,0.0,0.34,1.12,2.21,3.54,319.74,0.62,0.68,0.950000,1.390000,1.560000,387.55,1039.90,0.0,0.0,-843.37,6.64,8.33,0.0,0.0,0.0,0.0
276336,0.0,0.0,0.0,25.040,258.16,212.86,31.14,258.16,442.0,582.21,NaN,-1049.44,1090.45,50.0,26.2,0.000000,1.9,250.0,78.0,26.4,22.0,989.5,999.1,0.0,1.18,1.35,255.36,25.04,0.85,0.20,0.0,0.34,1.12,2.21,3.87,323.84,0.62,0.68,0.950000,1.390000,1.560000,454.91,1073.46,0.0,0.0,-1023.37,6.64,8.33,NaN,NaN,NaN,NaN


In [499]:
tot.sort_index(inplace=True)

## datetime 다시 만들어서 붙히기

In [500]:
df_water_copy.head(3)

,swl,inf,sfw,ecpc,tototf,tide_level,wl_1018662,fw_1018662,wl_1018680,fw_1018680,wl_1018683,fw_1018683,wl_1019630,fw_1019630
ymdhm,,,,,,,,,,,,,,
2012-05-01 00:00:00,24.800,555.0,219.07,24.93,555.0,445.0,310.7,469.05,300.2,0.0,290.0,729.80,275.3,540.18
2012-05-01 00:10:00,24.794,464.6,218.86,25.15,562.9,449.0,314.7,498.00,300.2,0.0,290.0,731.48,275.3,540.18
2012-05-01 00:20:00,24.789,478.1,218.69,25.31,576.4,451.0,313.7,490.68,301.2,0.0,290.0,726.42,275.3,540.18


In [501]:
df_rf_copy.reset_index(inplace=True)

In [502]:
df_datetime=df_rf_copy[['ymdhm']]

In [503]:
df_datetime.head(5)

,ymdhm
0,2012-05-01 00:00:00
1,2012-05-01 00:10:00
2,2012-05-01 00:20:00
3,2012-05-01 00:30:00
4,2012-05-01 00:40:00


In [504]:
# tot 이랑 시간 붙혀주기
tot=pd.concat([df_datetime, tot], axis=1)

In [505]:
# 첫행이랑 막행 없앰 =nan있는 행 삭제
tot=tot.iloc[1:-1]

In [506]:
tot

,ymdhm,rf_10184100,rf_10184110,rf_10184140,swl,inf,sfw,ecpc,tototf,tide_level,fw_1018662,fw_1018680,fw_1018683,fw_1019630,minute,ta,rn,ws,wd,hm,pv,td,pa,ps,팔당댐_wl,남양주시(팔당대교)_wl,서울시(너부대교)_fw,남양주시(팔당대교)_fw,남양주시(팔당댐)_wl,서울시(오금교)_wl,서울시(너부대교)_wl,남양주시(팔당댐)_fw,서울시(월계2교)_wl,서울시(광진교)_wl,서울시(대곡교)_wl,서울시(대곡교)_fw,서울시(광진교)_fw,서울시(월계2교)_fw,서울시(중랑교)_wl,서울시(창동교)_wl,서울시(대치교)_fw,서울시(대치교)_wl,서울시(청담대교)_fw,서울시(행주대교)_fw,팔당댐_fw,서울시(잠수교)_fw,서울시(한강대교)_fw,서울시(오금교)_fw,서울시(중랑교)_fw,wl_1018662,wl_1018680,wl_1018683,wl_1019630
1,2012-05-01 00:10:00,0.0,0.0,0.0,24.800,555.00,219.07,24.93,555.00,445.0,469.05,0.0,729.80,540.18,0.0,21.2,0.426115,2.4,110.0,43.0,10.8,8.1,1001.1,1011.0,0.0,1.56,1.12,497.47,24.81,1.06,0.17,0.0,0.44,1.20,1.78,7.67,2054.51,0.42,0.54,1.212996,2.713208,1.537687,469.05,540.18,0.0,0.0,729.80,6.71,3.88,314.7,300.2,290.0,275.3
2,2012-05-01 00:20:00,0.0,0.0,0.0,24.794,464.60,218.86,25.15,562.90,449.0,498.00,0.0,731.48,540.18,10.0,21.2,0.426115,2.4,110.0,43.0,10.8,8.1,1001.1,1011.0,0.0,1.56,1.12,497.47,24.81,1.06,0.17,0.0,0.44,1.20,1.78,7.67,2054.51,0.42,0.54,1.212996,2.713208,1.537687,469.05,540.18,0.0,0.0,729.80,6.71,3.88,313.7,301.2,290.0,275.3
3,2012-05-01 00:30:00,0.0,0.0,0.0,24.789,478.10,218.69,25.31,576.40,451.0,490.68,0.0,726.42,540.18,20.0,21.2,0.426115,2.4,110.0,43.0,10.8,8.1,1001.1,1011.0,0.0,1.56,1.12,497.47,24.81,1.06,0.17,0.0,0.44,1.20,1.78,7.67,2054.51,0.42,0.54,1.212996,2.713208,1.537687,498.00,540.18,0.0,0.0,731.48,6.71,3.88,311.7,301.2,290.0,276.3
4,2012-05-01 00:40:00,0.0,0.0,0.0,24.789,464.80,218.69,25.31,563.10,452.0,476.21,0.0,726.42,552.17,30.0,21.2,0.426115,2.4,110.0,43.0,10.8,8.1,1001.1,1011.0,0.0,1.56,1.12,497.47,24.80,1.06,0.17,0.0,0.43,1.20,1.78,7.67,2054.51,0.36,0.54,1.212996,2.713208,1.537687,490.68,540.18,0.0,0.0,726.42,6.71,3.88,311.7,301.2,291.0,277.3
5,2012-05-01 00:50:00,0.0,0.0,0.0,24.789,478.10,218.69,25.31,576.40,450.0,476.21,0.0,707.17,564.29,40.0,21.2,0.426115,2.4,110.0,43.0,10.8,8.1,1001.1,1011.0,0.0,1.56,1.12,497.47,24.80,1.06,0.17,0.0,0.44,1.20,1.78,7.67,2054.51,0.42,0.54,1.212996,2.713208,1.537687,476.21,552.17,0.0,0.0,726.42,6.71,3.88,311.7,301.2,291.0,277.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
276331,2022-07-18 23:10:00,0.0,0.0,0.0,25.040,865.96,212.86,31.14,259.30,528.0,319.84,NaN,25.27,926.70,0.0,26.2,0.000000,1.9,250.0,78.0,26.4,22.0,989.5,999.1,0.0,1.18,1.27,255.36,25.03,0.85,0.19,0.0,0.34,1.12,2.21,3.08,270.55,0.62,0.69,0.950000,1.390000,1.560000,325.72,865.00,0.0,0.0,329.00,6.64,8.66,0.0,0.0,0.0,0.0
276332,2022-07-18 23:20:00,0.0,0.0,0.0,25.040,259.23,212.86,31.14,259.23,510.0,319.84,NaN,-456.41,974.40,10.0,26.2,0.000000,1.9,250.0,78.0,26.4,22.0,989.5,999.1,0.0,1.18,1.35,255.36,25.03,0.85,0.20,0.0,0.34,1.12,2.21,3.13,282.85,0.62,0.69,0.950000,1.390000,1.560000,319.84,926.70,0.0,0.0,25.27,6.64,8.66,0.0,0.0,0.0,0.0
276333,2022-07-18 23:30:00,0.0,0.0,0.0,25.040,260.46,212.86,31.14,260.46,492.0,314.01,NaN,-717.30,1006.88,20.0,26.2,0.000000,1.9,250.0,78.0,26.4,22.0,989.5,999.1,0.0,1.18,1.35,255.36,25.03,0.85,0.20,0.0,0.34,1.12,2.21,2.77,315.64,0.62,0.68,0.950000,1.390000,1.560000,319.84,974.40,0.0,0.0,-456.41,6.64,8.33,0.0,0.0,0.0,0.0
276334,2022-07-18 23:40:00,0.0,0.0,0.0,25.040,259.37,212.86,31.14,259.37,475.0,387.55,NaN,-843.37,1039.90,30.0,26.2,0.000000,1.9,250.0,78.0,26.4,22.0,989.5,999.1,0.0,1.18,1.35,255.36,25.03,0.85,0.20,0.0,0.34,1.12,2.21,4.01,348.44,0.62,0.68,0.950000,1.390000,1.560000,314.01,1006.88,0.0,0.0,-717.30,6.64,8.33,0.0,0.0,0.0,0.0


In [507]:
# 4. valid 로 넘어가기
total=tot.copy()

## valid(우리들의 test), train 나누기

In [508]:
valid=total[(total['ymdhm']>'2022-04-30')&(total['ymdhm']<'2022-06-01')]
valid.reset_index(inplace=True, drop=True)

In [509]:
train=total[(total['ymdhm']<'2022-05-01')]
train.reset_index(inplace=True, drop=True)

In [510]:
# sklearn 랜포 다시 따라가는.
# 다시 분리
train_target = train.loc[:,submission.columns]
valid_target = valid.loc[:,submission.columns]

train_data = train.drop(submission.columns,axis=1)
valid_data = valid.drop(submission.columns,axis=1)
train_data=train_data.drop('ymdhm', axis=1)
valid_data=valid_data.drop('ymdhm', axis=1)

In [511]:
# null값 처리
train_target.fillna(train_target.mean(),inplace=True)
valid_target.fillna(train_target.mean(),inplace=True)
train_data.fillna(train_data.mean(),inplace=True)
valid_data.fillna(train_data.mean(),inplace=True)

In [512]:
print('--data--')
print(train_data.shape)
print(valid_data.shape)
print('--target--')
print(train_target.shape)
print(valid_target.shape)

--data--
(264959, 48)
(4464, 48)
--target--
(264959, 4)
(4464, 4)


## lgb 하나하나

In [513]:
base_params={'n_estimators': 900, 'num_leaves': 90, 'learning_rate': 0.1, 
             'colsample_bytree': 0.8, 'subsample': 0.9, 
             'reg_alpha': 5, 'reg_lambda': 7}

In [514]:
import lightgbm as lgb
from tqdm import tqdm

In [515]:
predictions = []
for i in tqdm(range(valid_target.shape[1])):
    base_model = lgb.LGBMRegressor(**base_params)
    base_model.fit(train_data, train_target.iloc[:, i])
    preds=base_model.predict(valid_data)
    predictions.append(preds)

100%|██████████| 4/4 [00:39<00:00,  9.90s/it]


In [516]:
tmp=pd.DataFrame(predictions[-4])
tmp['1']=pd.DataFrame(predictions[-3])
tmp['2']=pd.DataFrame(predictions[-2])
tmp['3']=pd.DataFrame(predictions[-1])

In [517]:
from sklearn.metrics import mean_squared_error
from math import sqrt

In [518]:
print('rmse : ', sqrt(mean_squared_error( valid_target.values, tmp.values ))) 

rmse :  2.88597908410363


# 찐테스트

In [519]:
train=total[(total['ymdhm']<'2022-06-01')]
train.reset_index(inplace=True, drop=True)
test=total[total['ymdhm']>='2022-06-01']
test.reset_index(inplace=True, drop=True)

In [520]:
# sklearn 랜포 다시 따라가는.
# 다시 분리
train_target = train.loc[:,submission.columns]
test_target = test.loc[:,submission.columns]

train_data = train.drop(submission.columns,axis=1)
test_data = test.drop(submission.columns,axis=1)
train_data=train_data.drop('ymdhm', axis=1)
test_data=test_data.drop('ymdhm', axis=1)

In [521]:
# null값 처리
train_target.fillna(train_target.mean(),inplace=True)
valid_target.fillna(train_target.mean(),inplace=True)
train_data.fillna(train_data.mean(),inplace=True)
valid_data.fillna(train_data.mean(),inplace=True)

In [523]:
print('--data--')
print(train_data.shape)
print(valid_data.shape)
print('--target--')
print(train_target.shape)
print(valid_target.shape)

--data--
(269423, 48)
(4464, 48)
--target--
(269423, 4)
(4464, 4)


In [524]:
# test
predictions = []
for i in tqdm(range(train_target.shape[1])):
    base_model = lgb.LGBMRegressor(**base_params)
    base_model.fit(train_data, train_target.iloc[:, i])
    preds=base_model.predict(test_data)
    predictions.append(preds)

100%|██████████| 4/4 [00:37<00:00,  9.36s/it]


In [525]:
sample['wl_1018662']=pd.DataFrame(predictions[-4])
sample['wl_1018680']=pd.DataFrame(predictions[-3])
sample['wl_1018683']=pd.DataFrame(predictions[-2])
sample['wl_1019630']=pd.DataFrame(predictions[-1])

In [526]:
sample.to_csv('ans_0902(lgb).csv', index=False)